# BAYER - VIGILOO --> Control de Interfaces y Consistencia de DATOS

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

def obtener_enlaces(url):
    # Realizar la petición HTTP a la página web
    respuesta = requests.get(url)
    # Si la petición fue exitosa, procesar el contenido
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.content, 'html.parser')
        # Extraer todos los elementos <a> de la página
        enlaces = soup.find_all('a', href=True)
        # Crear una lista para almacenar los datos
        datos = []
        # Recorrer cada enlace y almacenar la información necesaria
        for enlace in enlaces:
            # Extraer el href del enlace
            href = enlace['href']
            # Extraer el texto del enlace, que es el nombre del archivo
            texto = enlace.text
            # Intentar extraer las observaciones que están en el contenido antes del enlace
            # Esto asume que el texto de observaciones está justo antes del enlace
            observaciones = enlace.previous_sibling
            if observaciones:
                observaciones = observaciones.strip()  #Remueve los espacios al principio y al final de la línea
                # Dividir las observaciones en partes
                partes = observaciones.split("   ")
                #print(partes)
                # Extraer la fecha y la hora (primeras dos partes)
                fecha_hora = ' '.join(partes[:1])
                #print(fecha_hora)
                # Extraer el tamaño (última parte)
                tamaño = partes[-1]
                #print(tamaño)
            else:
                fecha_hora = "No disponible"
                tamaño = "No disponible"
            # Añadir a la lista como una tupla
            datos.append((texto, "http://rtv-b2b-api-logs.vigiloo.net" + href, fecha_hora, tamaño))    # datos.append((texto, url + href, fecha_hora, tamaño))
        # Convertir la lista en un DataFrame
        df = pd.DataFrame(datos, columns=['NombreArchivo', 'URL', 'FechaHora', 'Tamaño'])
        df.drop(0,axis=0)
        # Convertir la columna 'FechaHora' a tipo datetime
        df['FechaHora'] = pd.to_datetime(df['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p',errors='coerce') 
        # Convertir la columna 'Tamaño' a tipo numérico
        df['Tamaño'] = pd.to_numeric(df['Tamaño'], errors='coerce')
        df['Embarque'] = df['NombreArchivo'].str.extract(r'-(\d+)\.log')
        df.drop(0,axis=0, inplace=True)  # Borro la Primera línea que apunta al directorio padre        
        return df
    else:
        return "Error al acceder a la página"

### 1) Dataframe con el TOTAL de los archivos existentes de archivos de Log

In [2]:
# URL de la página de donde se desean extraer los enlaces
url = "http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/"
archivos_df = obtener_enlaces(url)
if isinstance(archivos_df, pd.DataFrame):
   print(archivos_df.head(1))
else:
   print("Error")

            NombreArchivo                                                URL  \
1  638270843928515554.log  http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...   

            FechaHora  Tamaño Embarque  
1 2023-08-08 08:39:00  2404.0      NaN  


In [3]:
# FILTRAR Archivos_df y generar subset en archivo SALIDA
# Inicio Nueva Nomenclatura de Archivos 01/18/2024

fecha_inicio = datetime.datetime(year=2024,month=1,day=18)

entrada = archivos_df[(archivos_df['FechaHora'] >=fecha_inicio)]
#salida.dropna()
# print(filtered_df)


In [4]:
entrada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5553 entries, 294 to 5846
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   NombreArchivo  5553 non-null   object        
 1   URL            5553 non-null   object        
 2   FechaHora      5553 non-null   datetime64[ns]
 3   Tamaño         5553 non-null   float64       
 4   Embarque       5546 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 260.3+ KB


## Explicación del Formato

%m: Representa el mes como un número con dos dígitos (01 a 12).

%d: Representa el día del mes como un número con dos dígitos (01 a 31).

%Y: Representa el año con cuatro dígitos.

%I: Representa la hora en formato de 12 horas (01 a 12).

%M: Representa los minutos.

%p: Representa AM o PM.

En nuestro caso como la fecha tiene los Dias y los Meses sin PADD hay que utilizar la siguiente opción

### pd.to_datetime(salida['FechaHora'], dayfirst=False).dt.strftime('%m/%d/%Y %I:%M %p')


https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

https://pandas.pydata.org/pandas-docs/version/1.0.1/reference/api/pandas.to_datetime.html

https://github.com/pandas-dev/pandas/issues/3341

In Linux "#" is replaced by "-":
%-d, %-H, %-I, %-j, %-m, %-M, %-S, %-U, %-w, %-W, %-y, %-Y
In Windows "-" is replaced by "#":
%#d, %#H, %#I, %#j, %#m, %#M, %#S, %#U, %#w, %#W, %#y, %#Y

#Linux
mydatetime.strftime('%-m/%d/%Y %-I:%M%p')
#Windows
mydatetime.strftime('%#m/%d/%Y %#I:%M%p')
Source: https://msdn.microsoft.com/en-us/library/fe06s4ak.aspx

## Procesar un registro JSON

* https://brightdata.es/blog/procedimientos/parse-json-data-with-python
* https://docs.python.org/3/tutorial/datastructures.html#dictionaries
* https://docs.python.org/3/tutorial/datastructures.html#more-on-lists


In [5]:
# Bisco un registro para probar

reg=2
url = entrada.iloc[reg]['URL']

print(url)
print('Archivo : ',entrada.iloc[reg]['NombreArchivo'])
print('Embarque: ',entrada.iloc[reg]['Embarque'])
print('Fecha   : ',entrada.iloc[reg]['FechaHora'])
print('Tamaño  : ',entrada.iloc[reg]['Tamaño'])
embarque = pd.read_json(url)

http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638411367567015104-833959623.log
Archivo :  638411367567015104-833959623.log
Embarque:  833959623
Fecha   :  2024-01-18 00:05:00
Tamaño  :  556.0


### Ejemplo de Uso

https://stackoverflow.com/questions/74118717/create-new-dataframe-with-json-objects-from-another-dataframe#:~:text=Solution%20using%20pandas&text=It%20provides%20a%20DataFrame%20object,dataframe%20with%20the%20original%20one



## Procesar El archivo SALIDA identificando los JSON

### PARAMETROS para la CONSULTA.

#### Comienza el proceso

In [6]:
# Extraer valores únicos de la columna 'Embarque' para usar en Query
embarques = entrada['Embarque'].unique()

# Formatear los valores entre apóstrofes y separados por comas
REGISTROS = ', '.join(f"'{embarque}'" for embarque in embarques)

#print(REGISTROS)

In [7]:
# PARAMETROS SI QUIERO CONSULTAS REGISTROS PUNTUALES --> SOBRE ESCRIBIR REGISTROS

REGISTROS=('834334133',
'834334229',
'834334503',
'834334858',
'834334859',
'834334860',
'834334871',
'834334872',
'834334888',
'834334898',
'834334903',
'834334904',
'834334912',
'834334918',
'834334923',
'834334928',
'834334937',
'834334938',
'834335052',
'834335056',
'834335092',
'834335094',
'834335538',
'834335546',
'834335551',
'834335559',
'834335572',
'834335581',
'834335615',
'834335617',
'834335631',
'834336739'
)


In [8]:
# FUNCIONES PARA LEER ARCHIVOS JSON
import requests
import json

def procesar_entregas_desde_url(url, modo='imprimir'):
    # Descargar el contenido del archivo JSON desde la URL
    respuesta = requests.get(url)
    respuesta.raise_for_status()  # Esto lanzará una excepción si ocurre un error

    # Leer el contenido JSON
    datos = respuesta.json()    
    resultados = []
    
    # Leer Datos de la Cabecera    
    resultado = f"\nOrigen: {datos['LogisticGroupCode']},  Embarque: {datos['Id']}, Date: {datos['Date']}, Carrier: {datos['CarrierName']}\n"
    resultados.append(resultado)
    # Imprimir la información si el modo es 'imprimir'
    if modo == 'imprimir':
        print(resultado)

    # Procesar y almacenar la información en una lista
    for stop in datos['StopList']:
        # Verificar que la parada tenga entregas
        if stop['OperationType'] == 'Dropped' and stop.get('Deliveries'):
            for delivery in stop['Deliveries']:
                delivery_number = delivery['DeliveryNumber']
                location = stop['LocationName']
                for item in delivery['Items']:
                    material_name = item['MaterialName']
                    batch = item.get('Batch', 'N/A')
                    qty = item['Qty']
                    
                    resultado = f"   DeliveryNumber: {delivery_number}, LocationName: {location}, MaterialName: {material_name}, Batch: {batch}, Qty: {qty}"
                    resultados.append(resultado)

                    # Imprimir la información si el modo es 'imprimir'
                    if modo == 'imprimir':
                        print(resultado)
    
    # Devolver los resultados si el modo es 'devolver'
    if modo == 'devolver':
        return resultados

    # Almacenar los resultados en un archivo si el modo es 'almacenar'
    if modo == 'almacenar':
        with open('resultados.txt', 'a', encoding="utf-8") as archivo_salida:
            for resultado in resultados:
                archivo_salida.write(resultado + '\n')
            archivo_salida.close()
                


# Ejemplo para validar el uso de la función
# url = 'http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638508724764465417-834195574.log'
# procesar_entregas_desde_url(url, modo='imprimir')


#### Explicación
* Leer el archivo JSON: Utilizamos json.load() para cargar el contenido del archivo JSON en una variable datos.
* Procesar las entregas: Iteramos sobre la lista de paradas (StopList). Para cada parada, obtenemos la lista de entregas (Deliveries).
* Extraer y mostrar la información: Para cada entrega, extraemos DeliveryNumber, LocationName, MaterialName, Batch, y Qty, y luego imprimimos esta información en el formato deseado.
* Función versátil: La función procesar_entregas permite imprimir, devolver o almacenar la información según el modo especificado.

##### Ejemplo de uso de la función
procesar_entregas('archivo.json', modo='imprimir')

Utiliza los PARAMETROS ingresados anteriormente

In [9]:
# FILTRADO y ORDENADO del dataframe para procesar secuencialmente

resumen = entrada.loc[entrada["Embarque"].isin(REGISTROS)]
salida = resumen.sort_values(by=['Embarque', 'FechaHora'])

In [10]:
# RUTINA PARA IMPRIMIR SALIDAS por la Terminal

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (REGISTROS):
        print('----------------------------------------------------------------')
        print('Embarque: ',salida.iloc[i]['Embarque'])
        url = salida.iloc[i]['URL']
        print(f'Archivo: ',salida.iloc[i]['FechaHora'],url)
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='imprimir')


----------------------------------------------------------------
Embarque:  834334133
Archivo:  2024-06-13 10:31:00 http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638538750654987980-834334133.log

Origen: SAPP08,  Embarque: 834334133, Date: 2024-06-13T16:45:00Z, Carrier: CH ROBINSON WORLDWIDE ARGENTINA SA

   DeliveryNumber: 5011517383, LocationName: LA BRAGADENSE SOCIEDAD ANONIMA, MaterialName: C LT LT721RR2 R9 80M T AR, Batch: WE8870004, Qty: 60.0
   DeliveryNumber: 5011517557, LocationName: LA BRAGADENSE SOCIEDAD ANONIMA, MaterialName: C LT LT3-44TRE C9 80M T AR, Batch: YE1091616, Qty: 100.0
   DeliveryNumber: 5011517539, LocationName: AIBAL SERVICIOS AGROPECUARIOS SOCIE, MaterialName: C DK DK73-03RR2 SPR 80M TB AR, Batch: YE1090945, Qty: 20.0
   DeliveryNumber: 5011517539, LocationName: AIBAL SERVICIOS AGROPECUARIOS SOCIE, MaterialName: C DK DK73-03TRE SPR 80M TB AR, Batch: YE1091215, Qty: 74.0
   DeliveryNumber: 5011517539, LocationName: AIBAL SERVICIOS AGROPECUARIOS SOCIE, Materi

In [11]:
# RUTINA PARA GRABAR en Archivo las SALIDAS 

with open('/data/resultados.txt', 'w', encoding="utf-8") as f:
    f.write(f"PROCESO DE INTEFACES desde resultado {fecha_inicio}" + '\n')
    f.close()

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (REGISTROS):
        with open('resultados.txt', 'a', encoding="utf-8") as f:
            f.write('\n----------------------------------------------------------------\n')
            f.write(f"Embarque: '{salida.iloc[i]['Embarque']}")
            url = salida.iloc[i]['URL']
            f.write(f"Archivo:  {salida.iloc[i]['FechaHora']} - {url}")
            f.close()
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='almacenar')

### Otras Alternativas para Analizar JSON Complejos

https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/python-jsonpath/#:~:text=Para%20que%20Python%20pueda%20leer,%E2%80%9D%20o%20%E2%80%9Cload()%E2%80%9D.



In [ ]:
salida.head(10)

In [12]:
# EXPORTAR los datos a EXCEL 
salida.to_excel("./salida.xlsx")

## 1) Generar Conexión con SQL SERVER

In [13]:
# CONECTAR con SERVIDOR SQL SERVER

import pyodbc
from dotenv import dotenv_values

secrets = dotenv_values(".env")
local_secrets = dotenv_values(".env.dev")

SERVER = secrets["SERVER"]
DATABASE = secrets["DATABASE"]
USERNAME = secrets["USERNAME"]
PASSWORD = secrets["PASSWORD"]
OPTIONS= secrets["OPTIONS"]

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};{OPTIONS}'
# print(connectionString)
conn = pyodbc.connect(connectionString) 

#    DRIVER={ODBC Driver 18 for SQL Server};SERVER=213.134.40.73,9595;DATABASE=seamtrack;UID=eduardo.ettlin@vigiloo.com.ar;PWD=Aladelta10$;encrypt=no
#    DRIVER={ODBC Driver 18 for SQL Server};SERVER=username;DATABASE=secret;UID=eduar;PWD=Aladelta10$;encrypt=no

## 2) Extraerlos Datos procesados que están en SQL SERVER

In [16]:
# Armar la Consulta con sus PARAMETROS

SQL_QUERY = f"""
SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN {REGISTROS}
ORDER BY A.RefId,D.RefId, D.[DestinationCode],E.[MaterialCode];
"""


In [17]:
print(SQL_QUERY)


SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN ('834328734', '834328569', '834328589', '834328740', '834328588', '834328894', '834328676', '834328912', '834328899', '834328893', '834328655', '834328674', '834329012', '834328851', '834328898', '834328675', '834328789', '834328796', '834328919', '834328735', '834328656', '834328785', '834328737', '834328741', '834328795', '834328562', '834328916', '834328451', '834328439', '834328849', '834327838', '834327903', '834327913', '834328437', '834328436', '

In [18]:
# Ejecutar la Consulta
cursor = conn.cursor()
cursor.execute(SQL_QUERY)


In [19]:
records = cursor.fetchall()

In [20]:
# IMPRIMIR Resultados de la Consulta
for r in records:
    print(f"{r.EMBARQUE}\t{r.DELIVERY}\t{r.FECHA}\t{r.DestinationCode}\t{r.DestinationName}\t{r.MaterialName}\t{r.Batch}\t{r.QTY}\t{r.LogisticGroupId}")

834327838	8480746909	2024-06-12 16:09:00	0003781342	AGROP.DEL PARANA SA	ALIETTE (RVTA CO) WP80 12X1KG BAG AR	LB00019244	    36	F21EE1F9-4823-42B4-AD40-8C81F2847818
834327838	8480747035	2024-06-12 16:09:00	0003775915	AGROSERVICIOS DEL NEA SRL	CROPSTAR FS600 4X5L BOT AR	LA40006642	     1	F21EE1F9-4823-42B4-AD40-8C81F2847818
834327838	8480747035	2024-06-12 16:09:00	0003775915	AGROSERVICIOS DEL NEA SRL	LATIUM SUPER (ZRT) EC360 2X10L BOT AR	LA47001180	   200	F21EE1F9-4823-42B4-AD40-8C81F2847818
834327838	8480747196	2024-06-12 16:09:00	0003780374	COOPERATIVA AGRICOLA MIXTA LA PROTE	LATIUM SUPER (ZRT) EC360 2X10L BOT AR	LA47001181	    43	F21EE1F9-4823-42B4-AD40-8C81F2847818
834327838	8480747488	2024-06-12 16:09:00	0003652563	AGRO GESTION DEL LITORAL SA	SCENIC FS80 4X5L BOT AR	LA40006773	     2	F21EE1F9-4823-42B4-AD40-8C81F2847818
834327838	8480747488	2024-06-12 16:09:00	0003652563	AGRO GESTION DEL LITORAL SA	CRIPTON XPRO SC450 2X10L BOT AR	LA40006678	     8	F21EE1F9-4823-42B4-AD40-8C81F284781

In [21]:
# Generar DATAFRAME con los resultados de las consultas
df = pd.DataFrame((tuple(t) for t in records)) 
df.columns = ['EMBARQUE','DELIVERY','FECHA','PEDIDO','DestinationCode','DestinationName','MaterialCode','MaterialName','Batch','QTY','LogisticGroupId']

In [22]:
# Eliminar Registros con datos Nulos
df.dropna(subset=['EMBARQUE','DELIVERY'], inplace=True)

In [23]:
# Adecuar el Grupo Logístico
df["LogisticGroupId"] = df["LogisticGroupId"].replace(
    { "75247427-34FD-41A3-B7E0-F0B8312EADA8": "Monsanto", "F21EE1F9-4823-42B4-AD40-8C81F2847818": "BAYER" })

In [24]:
df.head(20)

,EMBARQUE,DELIVERY,FECHA,PEDIDO,DestinationCode,DestinationName,MaterialCode,MaterialName,Batch,QTY,LogisticGroupId
0,834327838,8480746909,2024-06-12 16:09:00,8450207044,0003781342,AGROP.DEL PARANA SA,84404640,ALIETTE (RVTA CO) WP80 12X1KG BAG AR,LB00019244,36,BAYER
1,834327838,8480747035,2024-06-12 16:09:00,8450206853,0003775915,AGROSERVICIOS DEL NEA SRL,5820420,CROPSTAR FS600 4X5L BOT AR,LA40006642,1,BAYER
2,834327838,8480747035,2024-06-12 16:09:00,8450206853,0003775915,AGROSERVICIOS DEL NEA SRL,89311861,LATIUM SUPER (ZRT) EC360 2X10L BOT AR,LA47001180,200,BAYER
3,834327838,8480747196,2024-06-12 16:09:00,8450208921,0003780374,COOPERATIVA AGRICOLA MIXTA LA PROTE,89311861,LATIUM SUPER (ZRT) EC360 2X10L BOT AR,LA47001181,43,BAYER
4,834327838,8480747488,2024-06-12 16:09:00,8450197501,0003652563,AGRO GESTION DEL LITORAL SA,79258488,SCENIC FS80 4X5L BOT AR,LA40006773,2,BAYER
5,834327838,8480747488,2024-06-12 16:09:00,8450197501,0003652563,AGRO GESTION DEL LITORAL SA,86272768,CRIPTON XPRO SC450 2X10L BOT AR,LA40006678,8,BAYER
6,834327838,8480747488,2024-06-12 16:09:00,8450197501,0003652563,AGRO GESTION DEL LITORAL SA,88484649,LAUDIS SC630 4X5L BOT AR,LA47001052,7,BAYER
7,834327838,8480747488,2024-06-12 16:09:00,8450197501,0003652563,AGRO GESTION DEL LITORAL SA,89311861,LATIUM SUPER (ZRT) EC360 2X10L BOT AR,LA47001181,2,BAYER
8,834327838,8480747489,2024-06-12 16:09:00,8450206153,0003780932,GRUPO SAV ARGENTINA SA,89311861,LATIUM SUPER (ZRT) EC360 2X10L BOT AR,LA47001181,2,BAYER
9,834327838,8480747566,2024-06-12 16:09:00,8470206912,0003890362,TRES ROBLES S.R.L.,80897995,PUCARA FS400 12X1L BOT AR,LA40005612,60,BAYER


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   EMBARQUE         496 non-null    object        
 1   DELIVERY         496 non-null    object        
 2   FECHA            496 non-null    datetime64[ns]
 3   PEDIDO           496 non-null    object        
 4   DestinationCode  496 non-null    object        
 5   DestinationName  496 non-null    object        
 6   MaterialCode     496 non-null    object        
 7   MaterialName     496 non-null    object        
 8   Batch            244 non-null    object        
 9   QTY              496 non-null    object        
 10  LogisticGroupId  496 non-null    object        
dtypes: datetime64[ns](1), object(10)
memory usage: 42.8+ KB


In [26]:
# EXPORTAR DATOS EN FORMATO EXCEL

df.to_excel("/data/SQLServer.xlsx")

## CONTROL COMPLETO de TODAS LAS INTERFACES

#### Verificar si todo lo que está en LoadUpload EXISTE en las bases de VIGILOO

In [27]:
print(len(REGISTROS))

48


In [28]:
# Armar la Consulta con sus PARAMETROS

SQL_QUERY = f"""
SELECT DISTINCT     
       A.[RefId] as EMBARQUE
FROM [seamtrack].[vigiloo].[Shipment] A
"""

In [29]:
# Ejecutar la Consulta
cursor = conn.cursor()
cursor.execute(SQL_QUERY)

In [30]:
records = cursor.fetchall()

In [31]:
# Generar DATAFRAME con los resultados de las consultas
sqldf = pd.DataFrame((tuple(t) for t in records)) 
sqldf.columns = ['EMBARQUE']

In [32]:
sqldf.head()

,EMBARQUE
0,832146352
1,832174238
2,832194241
3,830627362
4,834266921


### Comparación de DATAFRAMES

#### 1) Archivos en Interfaces que NO ESTAN en la base de Datos. 

In [33]:
# UTILIZANDO LEFT JOIN
# Realizar un merge left con un indicador que muestre de dónde vienen los datos
merged_df = pd.merge(df, sqldf, on='EMBARQUE', how='left', indicator=True)

# Filtrar los registros que solo existen en el DataFrame 'df'
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Seleccionar solo las columnas del DataFrame original 'df'
result_df = result_df[df.columns]


In [34]:
print("Cantidad de INTEFACES NO PROCESADAS: ",len(result_df))

Cantidad de INTEFACES NO PROCESADAS:  0


In [35]:
result_df.head()

,EMBARQUE,DELIVERY,FECHA,PEDIDO,DestinationCode,DestinationName,MaterialCode,MaterialName,Batch,QTY,LogisticGroupId


In [36]:
# Crear una serie booleana que indica si cada valor de 'EMBARQUE' en 'df' está también en 'sqldf'
mask = df['EMBARQUE'].isin(sqldf['EMBARQUE'])

# Usar la negación de la máscara para filtrar registros
result_df = df[~mask]



#### 2) Embarque que estan en la Base de Datos y NO ESTAN en Interfaces

In [37]:
# UTILIZANDO JOIN RIGHT
# Realizar un merge right con un indicador que muestre de dónde vienen los datos
merged_df = pd.merge(df, sqldf, on='EMBARQUE', how='right', indicator=True)

# Filtrar los registros que solo existen en el DataFrame 'sqldf'
result_df = merged_df[merged_df['_merge'] == 'right_only']

# Seleccionar solo las columnas del DataFrame original 'sqldf'
result_df = result_df[sqldf.columns]

In [38]:
# ALTERNATIVA --> Verificando si está contenido
# Crear una serie booleana que indica si cada valor de 'EMBARQUE' en 'sqldf' está también en 'df'
mask = sqldf['EMBARQUE'].isin(df['EMBARQUE'])

# Usar la negación de la máscara para filtrar registros
result_df = sqldf[~mask]


In [39]:
print("Cantidad de EMBARQUE en la BD que no estan en INTEFACES: ",len(result_df))

Cantidad de EMBARQUE en la BD que no estan en INTEFACES:  15809


In [40]:
result_df.head()

,EMBARQUE
0,832146352
1,832174238
2,832194241
3,830627362
4,834266921
